## 0.1 Overview

Here, we prove that the RUP is indeed Rupert but not locally Rupert.

This document is structured in three parts:
    
1) [**Importing Stuff**](#1): We define a rational approximation of the RUP and import the csv's.
2) [**RUP is Rupert**](#2): We prove that the RUP is RUPERT with Nieuwland number at least 1.003.
3) [**RUP is not locally Rupert**](#3): In order to do so, we split the prove in three parts:
   - [**3.1 General solution-tree integrety**](#3.1): We make sure, that the tree does not have any obvious faults. We check, that the root of the tree is what it should be, that the union of the regions of every child is indeed the region of their parent. This is essentially it.
   - [**3.2 Defining Functions**](#3.2): We implement a lot of functions, that are despribed in the paper. All of these will have rational inputs and outputs! In the paper, we denoted rational functions with \tilde. However, as here all functions are rational, **we omit this notion to make everything better readable.** (Except the rational approximations of sin and cos)
   - [**3.3 Verifying**](#3.3): Description needed

## 0.2 Data Structure

Needs to be explained further

# 1. Importing Stuff  <a id="1"></a>

We start by importing dependencies. These are just: 
1. Sage-packages
2. The constants **INTERVAL_DENOMINATOR** and **kappa**
3. [**The RUP**](#1.1)
    -  The vertices of the RUP, stored in a 90 x 3 matrix called **poly**
    -  We choose a rational approximations of the vertices of the RUP, stored in a 90 x 3 matrix called **polyRat**, that is used throuout this script.
4. [**The Tables**](#1.2), stored as the dataframes **MAPPING** and **C** and 

In [1]:
import pandas as pd
from sys import getsizeof
import time
INTERVAL_DENOMINATOR = 10000
kappa=10^(-10)
PATH_mapping='./mapping.csv'
PATH_C_k='./C_k.csv'

In [2]:
## unimportant code - just for timekeeping:

def remainingTime(i,n,startTime):
    if i==0:
        i=1
    averageIterationTime=(time.time()-startTime)/i
    remainingTime=floor((n-i)*averageIterationTime)

    hours = remainingTime // 3600
    minutes = (remainingTime % 3600) // 60
    seconds = remainingTime % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

## 1.1 Defining the (rational) RUP <a id="1.1"></a>

First, we define the sexygon and fix its rational approximation

In [3]:
# the 3 points used to generate the Nopert and their norms:
C1=vector([3939,0 ,4340])/5861
C2=vector([7855,4178,4484])/10^4
C3=vector([9526,2057,1102])/10^4

normC1_squared=C1[0]^2+C1[1]^2+C1[2]^2
normC2_squared=C2[0]^2+C2[1]^2+C2[2]^2
normC3_squared=C3[0]^2+C3[1]^2+C3[2]^2

print(f"The point C1 has a norm² of {normC1_squared}")
print(f"The point C2 has a norm² of {normC2_squared} = {normC2_squared+0.0}... <1")
print(f"The point C3 has a norm² of {normC3_squared} = {normC3_squared+0.0}... <1")
print("As claimed, their maximal norm is indeed 1.")

The point C1 has a norm² of 1
The point C2 has a norm² of 19852593/20000000 = 0.992629650000000... <1
The point C3 has a norm² of 96190329/100000000 = 0.961903290000000... <1
As claimed, their maximal norm is indeed 1.


Calculating the vertices of the RUP and storing them as "poly":

In [4]:
# Rz(alpha) is the 3-dimensional rotation matrix around the z-axis as defined in the paper 
def Rz(alpha):
    return matrix([[cos(alpha),-sin(alpha),0],[sin(alpha),cos(alpha),0],[0,0,1]])

## We fill a 90 x 3 matrix with the vertices in the order described in the paper
poly= matrix(SR,90, 3) 
for i in range(3):
    for k in range(15):
        for l in range(2):
            if i==0:
                poly[k+15*i+45*l,:]=(-1)^l*Rz(2*pi*k/15)*C1
            if i==1:
                poly[k+15*i+45*l,:]=(-1)^l*Rz(2*pi*k/15)*C2
            if i==2:
                poly[k+15*i+45*l,:]=(-1)^l*Rz(2*pi*k/15)*C3
print("The first 5 points are")
pretty_print(poly[0:5,:])

The first 5 points are


[                                                             3939/5861                                                                      0                                                              4340/5861]
[3939/46888*sqrt(5) + 3939/23444*sqrt(-3/2*sqrt(5) + 15/2) + 3939/46888                                                 3939/5861*sin(2/15*pi)                                                              4340/5861]
[-3939/46888*sqrt(5) + 3939/23444*sqrt(3/2*sqrt(5) + 15/2) + 3939/46888                                                 3939/5861*sin(4/15*pi)                                                              4340/5861]
[                                       3939/23444*sqrt(5) - 3939/23444                                        3939/23444*sqrt(2*sqrt(5) + 10)                                                              4340/5861]
[3939/46888*sqrt(5) - 3939/23444*sqrt(-3/2*sqrt(5) + 15/2) + 3939/46888                                                 3939/5861*sin(7/15*pi)                                                              4340/5861]

Fixing the rational approximation of the RUP. It is kappa close.

In [5]:
polyRat=matrix(QQ,90,3)
## check the dokuentation of "floor"

for row in range(poly.nrows()):
    for col in range(poly.ncols()):
        polyRat[row,col]=floor(poly[row,col]*10^16)/10^16

print("The first 5 rational points are")
pretty_print(polyRat[0:5,:])

The first 5 rational points are


[ 1344139225388159/2000000000000000                                  0  3702439856679747/5000000000000000]
[  767457677370373/1250000000000000 2733553381804431/10000000000000000  3702439856679747/5000000000000000]
[ 1124255868643381/2500000000000000      39019144924547/78125000000000  3702439856679747/5000000000000000]
[ 1038404658627297/5000000000000000 6391761845566643/10000000000000000  3702439856679747/5000000000000000]
[  -87813004779749/1250000000000000 6683879450359373/10000000000000000  3702439856679747/5000000000000000]

## 1.2 Importing the csv's <a id="1.2"></a>

All nodes will be stored in a dataframe (?). Each node will store different parameters, that will be the columns of the table.

First we decide on which datatype to use for each column.

In [6]:
C=pd.read_csv(PATH_C_k, sep=',')
MAPPING=pd.read_csv(PATH_mapping, sep=',')

Importing the solution tree as 'TREE'

In [7]:
C

,ThetaMin,ThetaMax,PhiMin,PhiMax,P1_Q1_index,P2_Q2_index,P3_Q3_index,r
0,-6,258,-6,258,33,34,41,944
1,-6,126,252,390,30,31,38,951
2,-6,126,384,522,30,31,38,949
3,120,258,252,390,30,31,38,951
4,120,258,384,522,30,31,38,949
...,...,...,...,...,...,...,...,...
3530,4062,4200,15414,15516,34,42,50,750
3531,3930,4068,15510,15612,34,42,50,743
3532,3930,4068,15606,15714,34,42,50,737
3533,4062,4200,15510,15612,34,42,50,744


In [8]:
MAPPING

,i,j,C_index
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
1833295,699,2614,3534
1833296,699,2615,3534
1833297,699,2616,3534
1833298,699,2617,3534


# 2. RUP is Rupert  <a id="2"></a>

Here we prove, that RUP is Rupert with the solution given in the paper and Nieuwland constant at least 1.003.

To do so, we first calculate the inner and outer projections of the Polyhedron and scale the inner one by 1.003. These projections will be called P (inner Projection) and Q (outer Projection).

In order to prove that each point of P lies inside the convex hull of Q, we will for each points p of P find three points of Q, such that p lies inside these three points (as verifying that a point is inside a given triangle is faster than generally checking if it is inside the convex hull). To quickly find a promising tripple of Points of Q, we use rational approximations of P and Q.

In [9]:
def R_exact(alpha):
    A=matrix(SR,2,2)
    A[0,:]=matrix([cos(alpha),-sin(alpha)])
    A[1,:]=matrix([sin(alpha),cos(alpha)])
    return A

def M_exact(theta,phi):
    A=matrix(SR,2,3) ## 
    A[0,:]=matrix([-sin(theta),cos(theta),0])
    A[1,:]=matrix([-cos(theta)*cos(phi),-sin(theta)*cos(phi),sin(phi)])
    return A

def ScalarProduct_exact(vector1,vector2):
    ## returns the scalar product of two (vertical) vectors, aka n times 1 matrices
    
    assert vector1.ncols()==1, f"ScalarProduct wants integers or matrix as input"
    assert vector2.ncols()==1, f"ScalarProduct wants integers or matrix as input"
    assert vector1.nrows()==vector2.nrows(), f"Matrices must have the same number of rows"
    
    return (vector1.T*vector2)[0,0]

In [10]:
## Solution of the paper

theta1=  29/100
phi1  =  29/100
theta2=   2/100
phi2  = 227/100
alpha =-102/100
nieuwland=1003/1000

P=poly*M_exact(theta1,phi1).T*R_exact(alpha).T*nieuwland
Q=poly*M_exact(theta2,phi2).T


P_approx=matrix(QQ,90,2)
Q_approx=matrix(QQ,90,2)

for row in range(P.nrows()):
    for col in range(Q.ncols()):
        P_approx[row,col]=floor(P[row,col]*10^16)/10^16
        Q_approx[row,col]=floor(Q[row,col]*10^16)/10^16


In [11]:
## D is inside ABC iff (0,0) is inside the triange A-D,B-D,C-D
## some more explanation

def inequalityCheck(X,A,B,C):
    AA=A-X
    BB=B-X
    CC=C-X

    R_pi_2=matrix(QQ,[[0,-1],[1,0]]) ## is set to R(pi/2), i.e. rotation 90 degrees counter-clockwise
    if ScalarProduct_exact(R_pi_2*AA,BB)<=0:
        return False
    if ScalarProduct_exact(R_pi_2*BB,CC)<=0:
        return False
    if ScalarProduct_exact(R_pi_2*CC,AA)<=0:
        return False
        
    return True


In [12]:
## for each point of P, we want to find 3 points of Q, such that the point of P lies inside the triangle of Q
## to find promising three points of Q, we use rational approximations of the P and Q.
## Then, we do the check using the algebraic solver.

for i in range(90): ## iterate over the points of P, that have to be inside Q
    inside=False
    for j in range(90):
        if inside: break
        for k in range(90):
            if inside: break
            for l in range(90):
                if inside: break
                if (inequalityCheck(P_approx[i,:].T,Q_approx[j,:].T,Q_approx[k,:].T,Q_approx[l,:].T)):
                    ## Here we check, if the point Q[i] is indeed inside a triangle of P
                    if (inequalityCheck(P[i,:].T,Q[j,:].T,Q[k,:].T,Q[l,:].T)):
                        inside=True
                        print(f"Points P[{i}] is inside the triange Q[{j}], Q[{k}], Q[{l}]")
    assert inside, f"P[{i}] is outside of Q!"
print(f"We have proven, that RUP is Rupert with Nieuwland constant at least {nieuwland}!")

Points P[0] is inside the triange Q[0], Q[9], Q[48]
Points P[1] is inside the triange Q[0], Q[9], Q[46]
Points P[2] is inside the triange Q[0], Q[9], Q[45]
Points P[3] is inside the triange Q[0], Q[8], Q[59]
Points P[4] is inside the triange Q[0], Q[9], Q[72]
Points P[5] is inside the triange Q[0], Q[8], Q[71]
Points P[6] is inside the triange Q[0], Q[21], Q[86]
Points P[7] is inside the triange Q[0], Q[7], Q[85]
Points P[8] is inside the triange Q[0], Q[5], Q[32]
Points P[9] is inside the triange Q[0], Q[4], Q[1]
Points P[10] is inside the triange Q[0], Q[5], Q[1]
Points P[11] is inside the triange Q[0], Q[7], Q[1]
Points P[12] is inside the triange Q[0], Q[10], Q[1]
Points P[13] is inside the triange Q[0], Q[10], Q[2]
Points P[14] is inside the triange Q[0], Q[10], Q[22]
Points P[15] is inside the triange Q[0], Q[10], Q[47]
Points P[16] is inside the triange Q[0], Q[39], Q[45]
Points P[17] is inside the triange Q[0], Q[24], Q[59]
Points P[18] is inside the triange Q[0], Q[45], Q[57]


# 3. RUP is not locally Rupert  <a id="3"></a>

Here we follow the approach outlined in the paper: 
1. We have to prove, that there are no solutions in the sets $C_k\subset \mathbb{R}^5$ for $k=0,...,3534$ and
2. For each set $B_{i,j}$ there exists a set $C_k$ such that $B_{i,j} \subset C_k$.

The definition of the sets $B_{i,j}$ is outlined in the paper. The sets $C_k$ are stored in the datatable $C$. Its 
1. [**$B_{i,j}$ lie inside $C_k$**](#3.1): We prove that each B indeed lies inside one C
2. [**Defining Functions**](#3.2): Rational approximations of sine and cosine
3. [**Verifying the $C_k$**](#3.3): Many rotation/projection matrices

## 3.1 $B_{i,j}$ lie inside $C_k$<a id="3.1"></a>

In [13]:
startTime=time.time()

omega=6/10000
A=[omega*i for i in range(2700)]

for i in range(700):
    print(f"Processing: {i} of {699}; remaining: {remainingTime(i,699,startTime)}", end='\r')
    for j in range(2619):

        index=i*2619+j
        assert MAPPING["i"].iloc[int(index)]==i, f"There is a problem in row {index}"
        assert MAPPING["j"].iloc[int(index)]==j, f"There is a problem in row {index}"

        C_index=MAPPING["C_index"].iloc[int(index)]

        assert int(C["ThetaMin"].iloc[int(C_index)])/INTERVAL_DENOMINATOR<=A[i]-omega
        assert int(C["ThetaMax"].iloc[int(C_index)])/INTERVAL_DENOMINATOR>=A[i]+omega
        assert int(C["PhiMin"].iloc[int(C_index)])/INTERVAL_DENOMINATOR<=A[j]-omega
        assert int(C["PhiMax"].iloc[int(C_index)])/INTERVAL_DENOMINATOR>=A[j]+omega

KeyboardInterrupt: 

## 3.2 Defining Functions <a id="3.2"></a>

This part is three sections. The functions are exactly the same as in the other sage script.
   - [**3.2.1 Trigonometric functions**](#3.2.1): Rational approximations of sine and cosine
   - [**3.2.2 Defining Matrices**](#3.2.2): Many rotation/projection matrices
   - [**3.2.3 Miscellaneous functions**](#3.2.3): some clever stuff

## 3.2.1 Trigonometric functions <a id="3.2.1"></a>

In [16]:
def sinQ(x):
    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert abs(x)<=4, f"x should be between -4 and 4 and you gave me {x}"
    
    x2 = x^2
    
    return x*(1-x2/(2*3)*
             (1-x2/(4*5)*
             (1-x2/(6*7)*
             (1-x2/(8*9)*
             (1-x2/(10*11)*
             (1-x2/(12*13)*
             (1-x2/(14*15)*
             (1-x2/(16*17)*
             (1-x2/(18*19)*
             (1-x2/(20*21)*
             (1-x2/(22*23)*
             (1-x2/(24*25)))))))))))))

def cosQ(x):
    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert abs(x)<=4, f"x should be between -4 and 4 and you gave me {x}"
    
    x2 = x^2
  
    return (1-x2/(1*2)*
           (1-x2/(3*4)*
           (1-x2/(5*6)*
           (1-x2/(7*8)*
           (1-x2/(9*10)*
           (1-x2/(11*12)*
           (1-x2/(13*14)*
           (1-x2/(15*16)*
           (1-x2/(17*18)*
           (1-x2/(19*20)*
           (1-x2/(21*22)*
           (1-x2/(23*24)))))))))))))

**Example Code**: Let's approximate some trigonometic functions

In [72]:
print(f"sin(2)  = {sin(2).n(75)}...")
print(f"sinQ(2) = {sinQ(2).n(75)}... = {sinQ(2)}")
print(f"cos(2)  = {cos(2).n(75)}...")
print(f"cosQ(2) = {cosQ(2).n(75)}... = {cosQ(2)}")

sin(2)  = 0.9092974268256816953960...
sinQ(2) = 0.9092974268256816954083... = 176985682680149282/194640034667203125
cos(2)  = -0.4161468365471423869976...
cosQ(2) = -0.4161468365471423868320... = -61559114366058857/147926426347074375


## 3.2.2 Defining Matrices <a id="3.2.2"></a> 

In [17]:
def abs_matrix(M): ## every entry is replaced by its absolute value
    #if not isinstance(M, Matrix): #Check if it is a matrix
    #    raise TypeError("Input must be a SageMath Matrix")
    nrows = M.nrows()
    ncols = M.ncols()
    new_matrix = matrix(M.base_ring(), nrows, ncols) #Create an empty matrix with the same base ring
    for i in range(nrows):
        for j in range(ncols):
            new_matrix[i,j] = abs(M[i,j])
    return new_matrix

In [18]:
def X(theta,phi):
    assert isinstance(theta, Rational) or isinstance(theta, Integer), f"Give me rational inputs, please! You gave me {type(theta)}"
    assert isinstance(phi  , Rational) or isinstance(phi  , Integer), f"Give me rational inputs, please! You gave me {type(phi  )}"
    
    A=matrix(QQ,3,1)
    A[:,0]=matrix([[cosQ(theta)*sinQ(phi)],[sinQ(theta)*sinQ(phi)],[cosQ(phi)]])

    return A 

In [19]:
def M(theta,phi):
    assert isinstance(theta, Rational) or isinstance(theta, Integer), f"Give me rational inputs, please! You gave me {type(theta)}"
    assert isinstance(phi  , Rational) or isinstance(phi  , Integer), f"Give me rational inputs, please! You gave me {type(phi  )}"
    
    A=matrix(QQ,2,3)
    A[0,:]=matrix([-sinQ(theta),cosQ(theta),0])
    A[1,:]=matrix([-cosQ(theta)*cosQ(phi),-sinQ(theta)*cosQ(phi),sinQ(phi)])

    return A 

In [20]:
def R(alpha):
    assert isinstance(alpha, Rational) or isinstance(alpha, Integer), f"Give me rational inputs, please! You gave me {type(alpha)}"

    A=matrix(QQ,2,2)
    A[0,:]=matrix([cosQ(alpha),-sinQ(alpha)])
    A[1,:]=matrix([sinQ(alpha),cosQ(alpha)])

    return A

## 3.2.3 Miscellaneous functions <a id="3.2.3"></a>

In [21]:
def ScalarProduct(vector1,vector2):
    ## returns the scalar product of two (vertical) vectors, aka n times 1 matrices
    
    assert vector1.ncols()==1, f"ScalarProduct wants integers or matrix as input"
    assert vector2.ncols()==1, f"ScalarProduct wants integers or matrix as input"
    assert vector1.nrows()==vector2.nrows(), f"Matrices must have the same number of rows"
    
    assert vector1.base_ring()==ZZ or vector1.base_ring()==QQ, f"ScalarProduct wants integers or matrix as input"
    assert vector2.base_ring()==ZZ or vector2.base_ring()==QQ, f"ScalarProduct wants integers or matrix as input"

    return (vector1.T*vector2)[0,0]

In [22]:
def NormSquared(vector):
    ## returns the squared norm of a (vertical) vector, aka n times 1 matrix
    
    return ScalarProduct(vector,vector)

In [23]:
def f_lower(x):
    ## returns a non-negative rational value, that is smaller (or equal) to the square root of the input
    ## THE LAST 3 LINES OF THE FUNCTION SUFFICE TO SHOW IT'S FUNCTIONALITY
    
    ## We want to have a very small relative error and not have
    ## the numerators and denominators get unnecessarily large.
    ## Also, I want the denominator to be of the form 10^n
    
    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert x>=0, f"I really tried to find the square root of {x}, but I couldn't"

    if x==0:
        return 0
    
    
    #####################################################
    ## first we find the unique(!) integer a such that ##
    ## x*100^a in [10^20,10^22)                        ##
    #####################################################

    ## initial guess for a:
    a=-floor(log(float(x))/log(100.))+10
  
    while x*100^a < 10^20 or x*100^a >= 10^22: ## repeat until x*100^a in [10^20,10^22)
        if x*100^a < 10^20:
            a+=1
        if x*100^a >= 10^22:
            a-=1

    ## we now have x*100^a in [10^20,10^22)

    ##############################################
    ## we find the unique(!) positive integer b ##
    ## such that b^2<= x*100^a < (b+1)^2        ##
    ##############################################

      
    ## initial guess for b:
    b=floor(sqrt(float(x*100^a)))
  
    while b^2> x*100^a or x*100^a >= (b+1)^2:
        if b^2 > x*100^a:
            b-=1
        if (b+1)^2 <= x*100^a:
            b+=1
            
    ############################
    ## we are ready to return ##
    ############################

    ## as b²<= x*100^a, we have b/10^a<=sqrt(x)

    result=b/10^a

    ## a few 'asserts' to ensure that everything works properly: 
    assert result>=0, f"The approximation should be non-negative"
    assert result^2<=x, f"We got {result} as a lower approximation of the sqrt of {x}"
    assert isinstance(result, Rational) or isinstance(result, Integer), f"Something went wrong!"
    
    return result

In [24]:
def f_upper(x):
    ## returns a non-negative rational value, that is greater (or equal) to the square root of the input
    
    ## We want to have a very small relative error and not have
    ## the numerators and denominators get unnecessarily large.

    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert x>=0, f"I really tried to find the square root of {x}, but I couldn't"

    if x==0:
        return 0
    
    result=x/f_lower(x)

    assert result>=0, f"The approximation should be non-negative"
    assert result^2>=x, f"We got {result} as an upper approximation of the sqrt of {x}"
    assert isinstance(result, Rational) or isinstance(result, Integer), f"Something went wrong!"
    
    return result

**Example Code**: Let's approximate the sqaure-root of 3.141

In [81]:
print(f"The sqaure-root of 3.141 is {sqrt(3.141)}...")
print(f"Its lower approximation is  {f_lower(3141/1000)+0.0}... = {f_lower(3141/1000)}")
print(f"Its upper approximation is  {f_upper(3141/1000)+0.0}... = {f_upper(3141/1000)}")

The sqaure-root of 3.141 is 1.77228665852903...
Its lower approximation is  1.77228665850000... = 3544573317/2000000000
Its upper approximation is  1.77228665855806... = 2094000000/1181524439


## 3.3 Verifying the $C_k$ <a id="3.3"></a>

There are many statements that need to be verified to apply the strong theorem. Here we focus on that 'L-condition'. This condition is special, as it only depends on which 6 points of the polyhedron are chosen (P1,P2,P3,Q1,Q2,Q3) and no other parameters. Hence, we can verify this condition for all strong theorem, by only considering which unique sextuples of points are used.

In [27]:
C

,ThetaMin,ThetaMax,PhiMin,PhiMax,P1_Q1_index,P2_Q2_index,P3_Q3_index,r
0,-6,258,-6,258,33,34,41,944
1,-6,126,252,390,30,31,38,951
2,-6,126,384,522,30,31,38,949
3,120,258,252,390,30,31,38,951
4,120,258,384,522,30,31,38,949
...,...,...,...,...,...,...,...,...
3530,4062,4200,15414,15516,34,42,50,750
3531,3930,4068,15510,15612,34,42,50,743
3532,3930,4068,15606,15714,34,42,50,737
3533,4062,4200,15510,15612,34,42,50,744


In [26]:
startTime=time.time()


for row in C.itertuples():
    
    if row.Index%5==0:   
        print(f"Processing: strong Theorem {row.Index} of {C.shape[0]}; remaining: {remainingTime(row.Index,C.shape[0],startTime)}", end='\r')
    
    
    
    T1_min=Integer(row.ThetaMin)/INTERVAL_DENOMINATOR
    T1_max=Integer(row.ThetaMax)/INTERVAL_DENOMINATOR
    V1_min=Integer(row.PhiMin)/INTERVAL_DENOMINATOR
    V1_max=Integer(row.PhiMax)/INTERVAL_DENOMINATOR
    T2_min=Integer(row.ThetaMin)/INTERVAL_DENOMINATOR
    T2_max=Integer(row.ThetaMax)/INTERVAL_DENOMINATOR
    V2_min=Integer(row.PhiMin)/INTERVAL_DENOMINATOR
    V2_max=Integer(row.PhiMax)/INTERVAL_DENOMINATOR
    A_min =-omega
    A_max =omega

    t1=(T1_min+T1_max)/2
    v1=(V1_min+V1_max)/2
    t2=(T2_min+T2_max)/2
    v2=(V2_min+V2_max)/2
    a=(A_min+A_max)/2

    eps=max(T1_max-T1_min,V1_max-V1_min,T2_max-T2_min,V2_max-V2_min,A_max-A_min)/2
  
    P1_index=row.P1_Q1_index
    P2_index=row.P2_Q2_index
    P3_index=row.P3_Q3_index
    Q1_index=row.P1_Q1_index
    Q2_index=row.P2_Q2_index
    Q3_index=row.P3_Q3_index
  
    P1=polyRat[P1_index,:].T
    P2=polyRat[P2_index,:].T
    P3=polyRat[P3_index,:].T
    Q1=polyRat[Q1_index,:].T
    Q2=polyRat[Q2_index,:].T
    Q3=polyRat[Q3_index,:].T
    
    r=Integer(row.r)/1000 ## SOLLTE KONSTANTE SEIN
    sigma_P=0
    sigma_Q=0
    
    
    R_pi_2=matrix(QQ,[[0,-1],[1,0]]) ## is set to R(pi/2)
    
    X_1=X(t1,v1)
    X_2=X(t2,v2)
    
    M_t1_v1=M(t1,v1)
    M_t2_v2=M(t2,v2)

    r1=(R(a)*M_t1_v1*P1 -M_t2_v2*Q1)/2
    r2=(R(a)*M_t1_v1*P2 -M_t2_v2*Q2)/2
    r3=(R(a)*M_t1_v1*P3 -M_t2_v2*Q3)/2
    
    delta=4*kappa*2+max(f_upper(NormSquared(r1)),
                        f_upper(NormSquared(r2)),
                        f_upper(NormSquared(r3)))
    
    g=283/100*eps+2*eps^2
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### That s_p condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~

    assert (-1)^sigma_P*ScalarProduct(X_1,P1)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert (-1)^sigma_P*ScalarProduct(X_1,P2)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert (-1)^sigma_P*ScalarProduct(X_1,P3)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
  
    #~~~~~~~~~~~~~~~~~~~~~~~~~~  
    ### That s_q condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~  
  
    assert (-1)^sigma_Q*ScalarProduct(X_2,Q1)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert (-1)^sigma_Q*ScalarProduct(X_2,Q2)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert (-1)^sigma_Q*ScalarProduct(X_2,Q3)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
    ### Those many inequalities ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert ScalarProduct(R_pi_2*M_t1_v1*P1,M_t1_v1*P2)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t1_v1*P2,M_t1_v1*P3)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t1_v1*P3,M_t1_v1*P1)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    
    assert ScalarProduct(R_pi_2*M_t2_v2*Q1,M_t2_v2*Q2)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q2,M_t2_v2*Q3)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q3,M_t2_v2*Q1)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### Points are far from the origin ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert f_lower(NormSquared(M_t2_v2*Q1))>= r+142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert f_lower(NormSquared(M_t2_v2*Q2))>= r+142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert f_lower(NormSquared(M_t2_v2*Q3))>= r+142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## That rational inequality ###
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    for i in range(1,4):
        if i==1:
            Qi_Index=Q1_index
            Qi=Q1
        if i==2:
            Qi_Index=Q2_index
            Qi=Q2
        if i==3:
            Qi_Index=Q3_index
            Qi=Q3
    
        denom1=f_upper(NormSquared(M_t2_v2*Qi))+142/100*eps+3*kappa*2
    
    
        for A_index in range(polyRat.nrows()):
            if A_index==Qi_Index:
                continue
                
            A=polyRat[A_index,:].T

            nom1=ScalarProduct(M_t2_v2*Qi,M_t2_v2*Qi-M_t2_v2*A)
            nom2=9*kappa*2^2+(f_upper(NormSquared(Qi-A))+2*kappa)*(284/100*eps+2*eps^2)
            
            nom=nom1-nom2

            
            denom2=f_upper(NormSquared(M_t2_v2*Qi-M_t2_v2*A))+284/100*eps+6*kappa*2
      
            denom=denom1*denom2
      
            frac=nom/denom
      
            assert frac>=(448/100*eps+2*delta)/(2*r), f"So... it seems the strong Theorem cannot be applied"
            

KeyboardInterrupt: 